In [1]:
x=1

In [21]:
import scanpy as sc
import scib
import numpy as np

In [3]:
# import warnings
# warnings.simplefilter(action='ignore')

In [17]:
adata = sc.read('../../../pipeline/data/pp/totalvi_pbmc.h5ad')
adata

AnnData object with n_obs × n_vars = 647366 × 2000
    obs: 'sample_id', 'n_genes', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt', 'full_clustering', 'initial_clustering', 'Resample', 'Collection_Day', 'Sex', 'Age_interval', 'Swab_result', 'Status', 'Smoker', 'Status_on_day_collection', 'Status_on_day_collection_summary', 'Days_from_onset', 'Site', 'time_after_LPS', 'Worst_Clinical_Status', 'Outcome', 'patient_id', 'split0', 'split1', 'split2', 'split3', 'split4', '_scvi_labels', '_scvi_batch'
    var: 'feature_types', 'means', 'variances', 'residual_variances', 'highly_variable_rank', 'highly_variable_nbatches', 'highly_variable_intersection', 'highly_variable'
    uns: 'Site_colors', 'Status_on_day_collection_summary_colors', '_scvi_manager_uuid', '_scvi_uuid', 'hvg', 'initial_clustering_colors', 'leiden', 'neighbors', 'pca', 'umap'
    obsm: 'X_pca', 'X_pca_harmony', 'X_totalVI', 'X_umap', '_scvi_extra_categorical_covs', 'protein_expression'
    layers: 'co

In [18]:
adata.obsm['latent'] = adata.obsm['X_totalVI'].copy()
adata.write('../../../pipeline/data/pp/totalvi_pbmc.h5ad')

In [12]:
batch_key = "Site"
label_key = "initial_clustering"

In [13]:
sc.pp.subsample(adata, n_obs=10000)

In [14]:
adata

AnnData object with n_obs × n_vars = 10000 × 2000
    obs: 'sample_id', 'n_genes', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt', 'full_clustering', 'initial_clustering', 'Resample', 'Collection_Day', 'Sex', 'Age_interval', 'Swab_result', 'Status', 'Smoker', 'Status_on_day_collection', 'Status_on_day_collection_summary', 'Days_from_onset', 'Site', 'time_after_LPS', 'Worst_Clinical_Status', 'Outcome', 'patient_id', 'split0', 'split1', 'split2', 'split3', 'split4', '_scvi_labels', '_scvi_batch'
    var: 'feature_types', 'means', 'variances', 'residual_variances', 'highly_variable_rank', 'highly_variable_nbatches', 'highly_variable_intersection', 'highly_variable'
    uns: 'Site_colors', 'Status_on_day_collection_summary_colors', '_scvi_manager_uuid', '_scvi_uuid', 'hvg', 'initial_clustering_colors', 'leiden', 'neighbors', 'pca', 'umap'
    obsm: 'X_pca', 'X_pca_harmony', 'X_totalVI', 'X_umap', '_scvi_extra_categorical_covs', 'protein_expression'
    layers: 'cou

In [15]:
%%time
metrics = scib.metrics.metrics(
    adata, 
    adata, 
    batch_key=batch_key, 
    label_key=label_key, 
    embed='latent',
    ari_=True,
    nmi_=True,
    silhouette_=True,
    graph_conn_=True,
    isolated_labels_asw_=True,
    verbose=True,
)

/lustre/groups/ml01/workspace/anastasia.litinetskaya/mambaforge/envs/scib/lib/python3.10/site-packages/scib/metrics/metrics.py:293: DeprecationWarning: Call to deprecated function (or staticmethod) opt_louvain.
  res_max, nmi_max, nmi_all = opt_louvain(


Clustering...
resolution: 0.1, nmi: 0.38630316473009724
resolution: 0.2, nmi: 0.38630316473009724
resolution: 0.3, nmi: 0.38630316473009724
resolution: 0.4, nmi: 0.38630316473009724
resolution: 0.5, nmi: 0.38630316473009724
resolution: 0.6, nmi: 0.38630316473009724
resolution: 0.7, nmi: 0.38630316473009724
resolution: 0.8, nmi: 0.38630316473009724
resolution: 0.9, nmi: 0.38630316473009724
resolution: 1.0, nmi: 0.38630316473009724
resolution: 1.1, nmi: 0.38630316473009724
resolution: 1.2, nmi: 0.38630316473009724
resolution: 1.3, nmi: 0.38630316473009724
resolution: 1.4, nmi: 0.38630316473009724
resolution: 1.5, nmi: 0.38630316473009724
resolution: 1.6, nmi: 0.38630316473009724
resolution: 1.7, nmi: 0.38630316473009724
resolution: 1.8, nmi: 0.38630316473009724
resolution: 1.9, nmi: 0.38630316473009724
resolution: 2.0, nmi: 0.38630316473009724
optimised clustering against initial_clustering
optimal cluster resolution: 0.1
optimal score: 0.38630316473009724
NMI...
ARI...
Silhouette score.

In [16]:
metrics

,0
NMI_cluster/label,0.386303
ARI_cluster/label,0.000432
ASW_label,0.542877
ASW_label/batch,0.931413
PCR_batch,NaN
cell_cycle_conservation,NaN
isolated_label_F1,NaN
isolated_label_silhouette,0.487664
graph_conn,0.024654
kBET,NaN


In [22]:
mean_integ_metrics = np.mean([metrics[0][i] for i in ['graph_conn', 'ASW_label/batch']])
mean_bio_metrics = np.mean([metrics[0][i] for i in ['ASW_label', 'NMI_cluster/label', 'ARI_cluster/label', 'isolated_label_silhouette']])

In [23]:
overall_score = 0.4*mean_integ_metrics + 0.6*mean_bio_metrics

In [24]:
metrics.loc['batch', 0] = mean_integ_metrics
metrics.loc['bio', 0] = mean_bio_metrics
metrics.loc['overall', 0] = overall_score

In [25]:
metrics

,0
NMI_cluster/label,0.386303
ARI_cluster/label,0.000432
ASW_label,0.542877
ASW_label/batch,0.931413
PCR_batch,NaN
cell_cycle_conservation,NaN
isolated_label_F1,NaN
isolated_label_silhouette,0.487664
graph_conn,0.024654
kBET,NaN
